In [1]:
# downloading Selenium libraries 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# downloading BeautifulSoup
# pip install --upgrade bs4
# from bs4 import BeautifulSoup

import pandas as pd
import re
import time

In [2]:
# setting Selenium options 
options = Options()
# options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
# setting website that should be scraped 
driver.get("https://www.newyorkfed.org/research/policy/nowcast#/nowcast")

In [4]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(driver.find_element(By.XPATH, '//*[@id="cookieButton"]'))).click()

In [5]:
quarters_research = driver.find_elements(By.CLASS_NAME, "quarters-research")

In [6]:
scrapped_data = pd.DataFrame()

for q_pointer in quarters_research:
    href = q_pointer.get_property("href")
    print(f'Loading {href}')
    
    driver.get(href)
    q_pointer.click()
    
#     app_table = driver.find_elements(By.TAG_NAME, 'app-table')[1]
    
    # html_before = driver.page_source
    # soup_before = BeautifulSoup(html_before, 'html.parser')
        
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(driver.find_elements(By.XPATH, "//*[@class='highcharts-point']")[-1])).click()
    time.sleep(5)
    
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(driver.find_element(By.XPATH, "//*[@class='expand-collapse-font ng-tns-c35-2 ng-star-inserted']"))).click()
    time.sleep(5)

    # html = driver.page_source
    # soup = BeautifulSoup(html, 'html.parser')
        
    print(len(driver.find_elements(By.XPATH, '//*[@id="dataContainer"]/div/*')))

    # TODO: download_data function
    # Data flow label
    data_flow_label = driver.find_elements(By.XPATH, '//*[@id="dataContainer"]/div/*')[0].text
    print(f"INFO: {data_flow_label} processing")

    # Table header
    # driver.find_elements(By.XPATH, '//*[@id="dataContainer"]/div/*')[1].text
    header = [
        'Model Update',
        'Nowcast GDP Growth',
        'Release Date',
        'Data Series',
        'Actual',
        'Impact',
        'Formula',
        'Substitution items',
        'Units',
        'Reference Period',
        'Definition',
        'Source',
        'Note'
    ]

    # Iterating over rows
    data_flow = pd.DataFrame(columns=header)
    for i in range(2, len(driver.find_elements(By.XPATH, '//*[@id="dataContainer"]/div/*'))):

        collapsed_row = driver.find_elements(By.XPATH, '//*[@id="dataContainer"]/div/*')[i]

        # click to expand
        collapsed_row.click()

        expanded_row = collapsed_row.text.rstrip()
        row_items = expanded_row.split("\n")

        if len(row_items) == 2:
            # Model Update and Nowcast GDP Growth
            model_update, nowcast_gdp_growth = row_items
            print(f"Processing Model Updated on {model_update}\nNowcast GDP Growth: {nowcast_gdp_growth}")
            continue
        elif len(row_items) == 4:
            # Impact of Data Releases
            extracted_data = {
                'Data Series': row_items[0],
                'Actual': row_items[1],
                'Impact': row_items[2],
                'Note': re.search(r'Note:\s*(.*)', expanded_row).group(1).strip()
            }

        elif len(row_items) > 4:
            # Extract the data        
            extracted_data = {
                'Release Date': re.search(r'^\d{1,2}:\d{2}[AP]M \w{3} \d{1,2}', expanded_row, re.MULTILINE | re.DOTALL).group(0),
                'Data Series': row_items[1],
                'Actual': row_items[2],
                'Impact': row_items[3],
                'Formula': re.search(r'^(.*) = Impact', expanded_row.replace("\n".join(map(str, row_items[:4])), "").lstrip(), re.MULTILINE | re.DOTALL).group(0).replace("\n", " "),
                'Substitution items': re.search(r'Impact\n  (.*)\n  Units', expanded_row).group(1),
                'Units': re.search(r'Units:\s*(.*) Reference Period:', expanded_row).group(1),
                'Reference Period': re.search(r'Reference Period:\s*(.*)\n    Definition:', expanded_row).group(1),
                'Definition': re.search(r'Definition:\n(.*)\nSource:', expanded_row, re.MULTILINE | re.DOTALL).group(0).split("\n")[1],
                'Source': re.search(r'Source:\s*(.*)', expanded_row).group(1).strip(),

            }


        # Print the extracted data
        for key, value in extracted_data.items():
            print(f"{key}: {value}")
        print("-----")

        extracted_data.update({
            'Model Update': model_update,
            'Nowcast GDP Growth': nowcast_gdp_growth
        })

        data_flow = pd.concat([data_flow, pd.DataFrame(extracted_data, index=[0])]).reset_index(drop=True)
        data_flow['quarter_research'] = q_pointer.text
        
    scrapped_data = pd.concat([scrapped_data, data_flow]).reset_index(drop=True)

Loading https://www.newyorkfed.org/research/policy/nowcast#/nowcast/quarter/2024:Q3
101
INFO: Data Flow (Jul 26, 2024) processing
Processing Model Updated on Jul 26
Nowcast GDP Growth: 2.70
Release Date: 8:40AM Jul 26
Data Series: Real gross domestic product
Actual: 2.84
Impact: 0.15
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 2.84 2.04 0.19 0.15
Units: QoQ % chg. AR
Reference Period: Q2
Definition: Inflation-adjusted value of the goods and services produced in the economy less the value of the goods and services consumed in production over the quarter.
Source: U.S. Bureau of Economic Analysis
-----
Release Date: 8:30AM Jul 26
Data Series: PCE less food and energy: Chain price index
Actual: 0.18
Impact: -0.01
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.18 0.11 -0.13 -0.01
Units: MoM % chg.
Reference Period: Jun
Definition: Average change over the month in the prices paid for all domestic goods and services excluding food and energy.
Sourc

Data Series: Parameter revisions
Actual: 0.00
Impact: 0.00
Note: A measure of the impact on the nowcast of parameter reestimation, which occurs at the beginning of every quarter.
-----
Processing Model Updated on Jul 12
Nowcast GDP Growth: 2.17
Release Date: 8:33AM Jul 11
Data Series: CPI-U: All items less food and energy
Actual: 0.07
Impact: 0.03
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.07 0.18 -0.24 0.03
Units: MoM % chg.
Reference Period: Jun
Definition: Average change over the month in the prices paid by urban consumers for a fixed market basket of goods and services excluding food and energy products.
Source: U.S. Bureau of Labor Statistics
-----
Release Date: 8:33AM Jul 11
Data Series: CPI-U: All items
Actual: -0.06
Impact: 0.03
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: -0.06 0.05 -0.33 0.03
Units: MoM % chg.
Reference Period: Jun
Definition: Average change over the month in the prices paid by urban consumers for a fixed market

Release Date: 8:32AM Jun 27
Data Series: Merchant wholesalers: Total inventories
Actual: 0.55
Impact: 0.00
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.55 0.31 -0.01 0.00
Units: MoM % chg.
Reference Period: May
Definition: Value, at cost, of merchandise on hand and available for sale at the end of the month.
Source: U.S. Census Bureau
-----
Release Date: 8:31AM Jun 27
Data Series: Manufacturers’ new orders: Durable goods
Actual: 0.11
Impact: 0.00
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.11 -0.14 0.02 0.00
Units: MoM % chg.
Reference Period: May
Definition: Dollar value of new orders placed with manufacturing establishments for the month.
Source: U.S. Census Bureau
-----
Data Series: Data revisions
Actual: 0.00
Impact: 0.00
Note: A measure of the impact on the nowcast of adjustments to the historical values of input data.
-----
Data Series: Parameter revisions
Actual: 0.00
Impact: 0.00
Note: A measure of the impact on the nowcast of pa

Release Date: 10:54AM Jun 06
Data Series: Exports: Goods and services
Actual: 0.79
Impact: 0.00
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.79 0.77 0.02 0.00
Units: MoM % chg.
Reference Period: Apr
Definition: Value of goods and services exported over the month.
Source: U.S. Census Bureau
-----
Release Date: 10:01AM Jun 03
Data Series: Value of construction put in place
Actual: -0.12
Impact: 0.00
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: -0.12 0.20 0.01 0.00
Units: MoM % chg.
Reference Period: Apr
Definition: Dollar value of construction work done in the United States, including cost of labor, materials, architectural and engineering services, overhead, interest, taxes, and contractors’ profits.
Source: U.S. Census Bureau
-----
Release Date: 10:00AM Jun 04
Data Series: JOLTS: Total job openings
Actual: -296.00
Impact: -0.01
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: -296.00 151.73 0.00 -0.01
Units: Level chg. (th

Release Date: 8:30AM Jul 17
Data Series: Housing starts
Actual: 2.97
Impact: 0.01
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 2.97 1.79 0.01 0.01
Units: MoM % chg.
Reference Period: Jun
Definition: Number of permit registrations over the month for construction of new buildings (primarily residential), where construction begins with excavation of the foundation.
Source: U.S. Census Bureau
-----
Release Date: 9:18AM Jul 17
Data Series: Industrial production index
Actual: 0.64
Impact: 0.07
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.64 0.10 0.12 0.07
Units: MoM % chg.
Reference Period: Jun
Definition: Index measuring real output in the U.S. economy’s industrial sector during the month.
Source: Board of Governors of the Federal Reserve System
-----
Data Series: Data revisions
Actual: 0.00
Impact: 0.09
Note: A measure of the impact on the nowcast of adjustments to the historical values of input data.
-----
Data Series: Parameter revisions
Actu

...

Release Date: 8:48AM Nov 14
Data Series: CPI-U: All items
Actual: 0.05
Impact: 0.22
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.05 0.33 -0.76 0.22
Units: MoM % chg.
Reference Period: Oct
Definition: Average change over the month in the prices paid by urban consumers for a fixed market basket of goods and services.
Source: U.S. Bureau of Labor Statistics
-----
Release Date: 8:30AM Nov 16
Data Series: Export price index
Actual: -1.07
Impact: -0.02
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: -1.07 0.31 0.02 -0.02
Units: MoM % chg.
Reference Period: Oct
Definition: Index of the price of all goods and services produced domestically and sold abroad over the month.
Source: U.S. Bureau of Labor Statistics
-----
Release Date: 8:30AM Nov 16
Data Series: Import price index
Actual: -0.78
Impact: -0.03
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: -0.78 0.30 0.03 -0.03
Units: MoM % chg.
Reference Period: Oct
Definition: Index of t

Data Series: Data revisions
Actual: 0.00
Impact: -0.02
Note: A measure of the impact on the nowcast of adjustments to the historical values of input data.
-----
Data Series: Parameter revisions
Actual: 0.00
Impact: -0.01
Note: A measure of the impact on the nowcast of parameter reestimation, which occurs at the beginning of every quarter.
-----
Processing Model Updated on Oct 27
Nowcast GDP Growth: 2.79
Release Date: 8:43AM Oct 27
Data Series: Real gross domestic product
Actual: 4.88
Impact: 0.41
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 4.88 2.59 0.18 0.41
Units: QoQ % chg. AR
Reference Period: Q3
Definition: Inflation-adjusted value of the goods and services produced in the economy less the value of the goods and services consumed in production over the quarter.
Source: U.S. Bureau of Economic Analysis
-----
Release Date: 8:30AM Oct 27
Data Series: PCE less food and energy: Chain price index
Actual: 0.30
Impact: -0.01
Formula: (Actual − Forecast) × Weight = I

Release Date: 8:37AM Oct 12
Data Series: CPI-U: All items less food and energy
Actual: 0.32
Impact: -0.02
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.32 0.23 -0.19 -0.02
Units: MoM % chg.
Reference Period: Sep
Definition: Average change over the month in the prices paid by urban consumers for a fixed market basket of goods and services excluding food and energy products.
Source: U.S. Bureau of Labor Statistics
-----
Release Date: 8:37AM Oct 12
Data Series: CPI-U: All items
Actual: 0.40
Impact: -0.07
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.40 0.29 -0.70 -0.07
Units: MoM % chg.
Reference Period: Sep
Definition: Average change over the month in the prices paid by urban consumers for a fixed market basket of goods and services.
Source: U.S. Bureau of Labor Statistics
-----
Release Date: 8:31AM Oct 13
Data Series: Export price index
Actual: 0.74
Impact: 0.02
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.74 0.35 0.0

Release Date: 8:30AM Sep 27
Data Series: Manufacturers’ inventories: Durable goods
Actual: 0.22
Impact: 0.00
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.22 0.10 -0.02 0.00
Units: MoM % chg.
Reference Period: Aug
Definition: Dollar value of manufacturers’ stocks of durable goods at the end of the month.
Source: U.S. Census Bureau
-----
Release Date: 8:30AM Sep 27
Data Series: Manufacturers’ unfilled orders: All industries
Actual: 0.38
Impact: 0.00
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.38 0.44 0.03 0.00
Units: MoM % chg.
Reference Period: Aug
Definition: Dollar value of unfilled orders placed with manufacturing establishments during the month.
Source: U.S. Census Bureau
-----
Release Date: 8:30AM Sep 27
Data Series: Manufacturer’s shipments: Durable goods
Actual: 0.51
Impact: -0.01
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 0.51 0.63 0.10 -0.01
Units: MoM % chg.
Reference Period: Aug
Definition: Dollar value 

Release Date: 8:31AM Sep 06
Data Series: Exports: Goods and services
Actual: 1.58
Impact: 0.03
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 1.58 0.58 0.03 0.03
Units: MoM % chg.
Reference Period: Jul
Definition: Value of goods and services exported over the month.
Source: U.S. Census Bureau
-----
Data Series: Data revisions
Actual: 0.00
Impact: -0.01
Note: A measure of the impact on the nowcast of adjustments to the historical values of input data.
-----
Data Series: Parameter revisions
Actual: 0.00
Impact: 0.01
Note: A measure of the impact on the nowcast of parameter reestimation, which occurs at the beginning of every quarter.
-----
Processing Model Updated on Sep 01
Nowcast GDP Growth: 2.21
Release Date: 10:01AM Sep 01
Data Series: ISM mfg.: Employment index
Actual: 48.50
Impact: 0.07
Formula: (Actual − Forecast) × Weight = Impact
Substitution items: 48.50 45.25 0.02 0.07
Units: Index
Reference Period: Aug
Definition: Diffusion index based on a survey of U.S. 

In [8]:
# TODO: weights

In [9]:
scrapped_data

,Model Update,Nowcast GDP Growth,Release Date,Data Series,Actual,Impact,Formula,Substitution items,Units,Reference Period,Definition,Source,Note,quarter_research
0,Jul 26,2.70,8:40AM Jul 26,Real gross domestic product,2.84,0.15,(Actual − Forecast) × Weight = Impact,2.84 2.04 0.19 0.15,QoQ % chg. AR,Q2,Inflation-adjusted value of the goods and serv...,U.S. Bureau of Economic Analysis,NaN,2024:Q3 |
1,Jul 26,2.70,8:30AM Jul 26,PCE less food and energy: Chain price index,0.18,-0.01,(Actual − Forecast) × Weight = Impact,0.18 0.11 -0.13 -0.01,MoM % chg.,Jun,Average change over the month in the prices pa...,U.S. Bureau of Economic Analysis,NaN,2024:Q3 |
2,Jul 26,2.70,8:30AM Jul 26,PCE: Chain price index,0.08,-0.07,(Actual − Forecast) × Weight = Impact,0.08 -0.01 -0.77 -0.07,MoM % chg.,Jun,Measure of the average change over the month i...,U.S. Bureau of Economic Analysis,NaN,2024:Q3 |
3,Jul 26,2.70,8:30AM Jul 26,Real disposable personal income,0.10,-0.03,(Actual − Forecast) × Weight = Impact,0.10 0.27 0.19 -0.03,MoM % chg.,Jun,Real value of the income available to persons ...,U.S. Bureau of Economic Analysis,NaN,2024:Q3 |
4,Jul 26,2.70,8:40AM Jul 26,Real personal consumption expenditures,0.22,-0.01,(Actual − Forecast) × Weight = Impact,0.22 0.24 0.41 -0.01,MoM % chg.,Jun,Value of the goods and services purchased duri...,U.S. Bureau of Economic Analysis,NaN,2024:Q3 |
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,Sep 01,2.21,8:30AM Aug 30,Merchant wholesalers: Total inventories,-0.09,0.00,(Actual − Forecast) × Weight = Impact,-0.09 -0.13 0.00 0.00,MoM % chg.,Jul,"Value, at cost, of merchandise on hand and ava...",U.S. Census Bureau,NaN,2023:Q4
708,Sep 01,2.21,10:01AM Aug 29,JOLTS: Total job openings,-338.00,-0.02,(Actual − Forecast) × Weight = Impact,-338.00 197.28 0.00 -0.02,Level chg. (thousands),Jul,Number of positions that are open (not filled)...,U.S. Bureau of Labor Statistics,NaN,2023:Q4
709,Sep 01,2.21,8:30AM Aug 30,Real gross domestic income,0.49,0.00,(Actual − Forecast) × Weight = Impact,0.49 0.81 0.01 0.00,QoQ % chg. AR,Q2,"Sum of incomes, minus inflation, earned in the...",U.S. Bureau of Economic Analysis,NaN,2023:Q4
710,Sep 01,2.21,NaN,Data revisions,0.00,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,A measure of the impact on the nowcast of adju...,2023:Q4
